# Cosmic-Ray Rejection

This time we will learn how to reject, i.e., erase, the unwanted signals: Cosmic-rays (CR). CR is a loosely defined term in observational astronomy used to describe high energetic particles (or the resulting radiation) originated from extraterrestrial sources. On CCD image, CR appears as a dot, short streak, as shown in the following image (image credit [Tony Hallas](http://astronomy.com/magazine/tony-hallas/2010/07/tony-hallas-cosmic-imaging-getting-clean-images)):

![CRexample](http://astronomy.com/-/media/import/images/1/e/e/september-2010-cosmic-rays.jpg?mw=600)



## 1. Introduction

The most widely used algorithm for CR rejection is the L.A.Cosmic by [Peter G. van Dokkum](http://docs.astropy.org/en/stable/io/fits/index.html). It uses the fact that CR has sharp edges and high pixel values, so that they appear differently compared to stellar sources after median filtering. 

Unfortunately, however, this is very slow in IRAF, especially for larger file size. This is because IRAF does not support multiprocessing, which is useful in many modern computation. I myself have a computer with 8 cores, and IRAF can only use up to 1 core, and 7 others are not working for CR rejection.

Curtis McCully has thus developed a C-based, but can be used in Python as if it is a totally-pythonic code, and it is named as [`astroscrappy`](https://github.com/astropy/astroscrappy). It is about 20-40 times faster than pure python codes, and up to about 100 times faster than IRAF implementation. The number can even increase if you use faster algorithms using optional arguments of `astroscrappy`.

## 2. Download Astroscrappy

You can download it via `conda`:

     conda install -c astropy astroscrappy=1.0.5 

For check, you can turn IPython on and type

```python
import astroscrappy
astroscrappy.test()
```

## 3. Usage

CR can be everywhere. Even on bias image, which should have 0-second exposure, you may see CR, because the instrument cannot have mathematically zero exposure (e.g., 0.001 second). While reading out bias image, a CR may hit CCD pixel and it will appear on bias image. On dark or flat images, which have exposure time of up to about several minutes, CR becomes extremely common. 

> But do we have to do CR rejection for all the images?

No. First of all, it takes too much time. Secondly, if we do median combine to many of the bias, dark, and flat images, CR-hit pixels (the pixels hit by CR) can be *clipped* without applying CR rejection algorithm.

For object images, however, all object images may not be identical, so we don't do median combine. That's why we need CR rejection to object images. **You can do CR rejection either before or after preprocessing** (I currently have no firm idea when is the best time to do this, so your opinion is welcomed). It's very simple to do such rejection: import and use.

Since the `astroscrappy` has default settings different from classic `L.A.Cosmic`, `ccdproc` has a function which does the classic `L.A.cosmic` by using `astroscrappy`.

There is no online manual of `astroscrappy`, but you can print it out in Python:

In [1]:
import astroscrappy
print(astroscrappy.detect_cosmics.__doc__)

detect_cosmics(indat, inmask=None, sigclip=4.5, sigfrac=0.3, objlim=5.0,
                      gain=1.0, readnoise=6.5, satlevel=65536.0, pssl=0.0,
                      niter=4, sepmed=True, cleantype='meanmask',
                      fsmode='median', psfmodel='gauss', psffwhm=2.5,
                      psfsize=7, psfk=None, psfbeta=4.765, verbose=False)

    Detect cosmic rays in a numpy array.

    If you use this code, please add this repository address in a footnote:
    https://github.com/astropy/astroscrappy

    Please cite the original paper which can be found at:
    http://www.astro.yale.edu/dokkum/lacosmic/

    van Dokkum 2001, PASP, 113, 789, 1420
    (article : http://adsabs.harvard.edu/abs/2001PASP..113.1420V)

    Parameters
    ----------
    indat : float numpy array
        Input data array that will be used for cosmic ray detection.

    inmask : boolean numpy array, optional
        Input bad pixel mask. Values of True will be ignored in the cosmic ray
        det

In [ ]:
import astroscrappy

obj = CCDData.read('HST_Tutorial/Pluto.fits', unit='adu')
obj_LA = obj.copy()
obj_cr = obj.copy()
# Following should give identical result to IRAF L.A.Cosmic,
# where "m_LA" is the mask image
m_LA, obj_LA.data = astroscrappy.detect_cosmics(obj.data,
                                                inmask=None,
                                                satlevel=np.inf,
                                                sepmed=False,
                                                cleantype='medmask',
                                                fsmode='median')
obj_LA.write('LA_'+objname)  

# Following is the "fastest" version. The author argues that this
# method gave comparable result as default L.A.Cosmic, but 100 times faster
m_cr, obj_cr.data = astroscrappy.detect_cosmics(obj.data)
obj_cr.write('cr_'+objname, overwrite=True)
